In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
plt.ioff();
try:
    maindir
except NameError:
    maindir = os.path.dirname(os.getcwd())
for module in os.listdir(maindir + '\\testing_modules'):
    %run -i "{maindir}\\testing_modules\\{module}"
base = os.path.dirname(os.path.dirname(maindir)) + '\\'
plt.ion();
np.set_printoptions(suppress=True)

In [19]:
%matplotlib qt
import skimage.io as io
import pyFAI
from skimage.restoration import rolling_ball
from lmfit.models import PolynomialModel, ExponentialModel, SplineModel
import matplotlib
from matplotlib.widgets import Button, Slider
from collections import OrderedDict
import xrayutilities as xu
from xrayutilities.materials import elements

In [2]:
scanid = 149566
filedir = '''D:\\Musterman_postdoc\\20231109_Musterman\\'''
#filedir = '''D:\\Musterman_postdoc\\20231030_Musterman\\'''
filedir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Data\\Condensed Datasets\\'''
map_filename = f'scan{scanid}_dexela_xrd.tif'
h5_filename = f'scan{scanid}_xrd.h5'

test = XRDMap.from_h5(h5_filename, wd=filedir)


In [14]:
cif_dir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Literature\\CIF\\'''
test.clear_phases()
test.load_phase('Barker\\1521772COD.cif', filedir=cif_dir, phase_name="LiNbO3 161")
test.load_phase('Barker\\LiNbO3.cif', filedir=cif_dir, phase_name="LiNbO3 148")
test.load_phase('AMCSD\\Platinum_0011157.cif', filedir=cif_dir, phase_name="Platinum")
test.load_phase('AMCSD\\Calcite_0000984.cif', filedir=cif_dir, phase_name="Calcite")
test.load_phase('AMCSD\\Aluminum_0011136.cif', filedir=cif_dir, phase_name="Aluminum")
test.load_phase('AMCSD\\Zincite_0011555.cif', filedir=cif_dir, phase_name="Zincite")
test.load_phase('AMCSD\\Gold_0011140.cif', filedir=cif_dir, phase_name="Gold")
test.load_phase('AMCSD\\Chromium_0011209.cif', filedir=cif_dir, phase_name="Chromium BCC")
test.load_phase('AMCSD\\Chromium_0011170.cif', filedir=cif_dir, phase_name="Chromium HCP")
test.load_phase('AMCSD\\Chromium_0011144.cif', filedir=cif_dir, phase_name="Chromium FCC")

XU.materials: Warning: element O used instead of O-2
XU.materials: Warning: element Nb used instead of Nb+5
XU.materials: Warning: element Li used instead of Li+1


In [6]:
test.select_phases(remove_less_than=0.1)

In [3]:
f = h5py.File(test.h5, 'r')

In [4]:
f['xrdmap']['phase_list']['Calcite']['WyckoffBase']['O'].attrs.keys()

<KeysViewHDF5 ['number', 'position']>

In [17]:
f['xrdmap']['phase_list']['Calcite'].attrs['space_group']

'167:H'

In [60]:
type(calcite.lattice._wbase[2][1][1][0])

numpy.float64

In [ ]:
np.array([None, None, ])

In [40]:
wbase[atom_key][0]

nan

In [ ]:
wbase = f['/xrdmap/phase_list/Calcite']['WyckoffBase']
atom_lst = []
for atom_key in wbase.keys():
    atom = Atom(atom_key, wbase[atom_key].attrs['number'])
    #atom = Atom(atom_key, atom_dict[atom_key])
    pos = wbase[atom_key].attrs['position']
    #positions = wbase[atom_key][:]
    if np.isnan(wbase[atom_key][0]):
        positions = None
    else:
        positions = wbase[atom_key][:]
    atom_lst.append((atom, (pos, *positions)))

In [78]:
f['/xrdmap/phase_list'].keys()

<KeysViewHDF5 ['Calcite', 'LiNbO3 161']>

In [80]:
new_lattice = load_from_h5(f['/xrdmap/phase_list/LiNbO3 161'])

#new_lattice = xu.materials.spacegrouplattice.SGLattice('167:H', 4.988, 17.061, atoms=[elements.Ca, elements.C, elements.O],
#                                                                                pos=[('6b', (None,)), ('6a', (None,)), ('18e', (0.2567,))])

In [81]:
new_phase = Phase('Calcite', lat = new_lattice)

In [82]:
new_phase.lattice._wbase

[(Li ( 3), ('6a', (0.25,)), 1.0, 0.0),
 (Nb (41), ('6a', (0.0,)), 1.0, 0.0),
 (O ( 8), ('18b', (0.041, 0.3333, 0.0833)), 1.0, 0.0)]

In [83]:
new_phase.get_hkl_reflections(energy=15000)

In [45]:
new_lattice._parameters

OrderedDict([('a', 4.988),
             ('b', 4.988),
             ('c', 17.061),
             ('alpha', 90),
             ('beta', 90),
             ('gamma', 120)])

In [34]:
atom_dict = {'Ca' : 20, 
             'C' : 6,
             'O' : 8}

In [5]:
from xrayutilities.materials.atom import Atom
from xrayutilities.materials.spacegrouplattice import sgrp_sym, SGLattice

In [79]:
def load_from_h5(group):
    name = group.name.split('/')[-1]
    
    params = OrderedDict()
    key_lst = ['a', 'b', 'c', 'alpha', 'beta', 'gamma']
    for key in key_lst:
        params[key] = group.attrs[key]
    
    space_group = group.attrs['space_group']
    space_group_number = group.attrs['space_group_number']

    wbase = group['WyckoffBase']
    atom_lst = []
    for atom_key in wbase.keys():
        atom = Atom(atom_key, wbase[atom_key].attrs['number'])
        #atom = Atom(atom_key, atom_dict[atom_key])
        pos = wbase[atom_key].attrs['position']
        if np.any(np.isnan(wbase[atom_key][0])):
            positions = None
        else:
            positions = (*wbase[atom_key][:][0],)
        atom_lst.append((atom, (pos, positions)))

    args = get_sym_args(space_group, params)
    lattice = SGLattice(space_group, *args, atoms=[atom[0] for atom in atom_lst],
                                                pos=[atom[1] for atom in atom_lst])
    
    return lattice
    

In [15]:
calcite = test.phases['Calcite']

In [108]:
phase_grp = f['xrdmap/phase_list']
iphase = phase_grp.require_group('calcite')

In [109]:
calcite.lattice._parameters

OrderedDict([('a', 4.988),
             ('b', 4.988),
             ('c', 17.061),
             ('alpha', 90),
             ('beta', 90),
             ('gamma', 120)])

In [142]:
for key, value in calcite.lattice._parameters.items():
    iphase.attrs[key] = value

iphase.attrs['space_group'] = calcite.lattice.space_group
iphase.attrs['space_group_number'] = sgrp_sym[int(calcite.lattice.space_group.split(':')[0])][0]

wbase = iphase.require_group('WyckoffBase')
for atom in calcite.lattice._wbase:
    data = np.array(atom[1][1:])
    data = np.array([np.nan for pos in data if pos is None])
    dset = wbase.require_dataset(atom[0].name, data=data, shape=data.shape, dtype=data.dtype)
    wbase.attrs['position'] = atom[1][0]

In [141]:
data.dtype

dtype('float64')

In [130]:
atom[1][1:]

(None,)

In [110]:
iphase.attrs['a'] = 4.988
iphase.attrs['b'] = 4.988
iphase.attrs['c'] = 17.061
iphase.attrs['alpha'] = 90
iphase.attrs['beta'] = 90
iphase.attrs['gamma'] = 120
iphase.attrs['space_group'] = 

In [15]:
calcite = test.phases['Calcite']
linbo = test.phases['LiNbO3 161']

In [146]:
test.phases

{'LiNbO3 161': LiNbO3 161 crystal phase class,
 'Calcite': Calcite crystal phase class}

In [97]:
linbo.lattice._wbase[0][0].name

'O'

In [112]:
calcite.lattice.space_group

'167:H'

In [81]:
params = calcite.lattice._parameters

In [33]:
calcite.lattice._wbase

[(Ca (20), ('6b', None), 1, 0.0),
 (C ( 6), ('6a', None), 1, 0.0),
 (O ( 8), ('18e', (0.2567,)), 1, 0.0)]

In [125]:
calcite.lattice._wbase[0][1]

('6b', None)

In [90]:
from xrayutilities.materials import elements

new_lattice = xu.materials.spacegrouplattice.SGLattice('167:H', 4.988, 17.061, atoms=[elements.Ca, elements.C, elements.O],
                                                                                pos=[('6b', None), ('6a', None), ('18e', (0.2567,))])

In [63]:
from xrayutilities.materials.wyckpos import RangeDict

sgrp_sym = RangeDict({range(1, 3): ('triclinic', 6),
                      range(3, 16): ('monoclinic', 4),
                      range(16, 75): ('orthorhombic', 3),
                      range(75, 143): ('tetragonal', 2),
                      range(143, 168): ('trigonal', 2),
                      range(168, 195): ('hexagonal', 2),
                      range(195, 231): ('cubic', 1)})

In [143]:
from xrayutilities.materials.spacegrouplattice import sgrp_sym

In [83]:
param[key] for key in []

KeyError: ('a', 'b')

In [7]:
# I dislike that I need this, but the SGLattice class can only handle the specified number of inputs
def get_sym_args(sgrp, params):

    sgrp = str(sgrp).lower()
    sgrp = sgrp.split(':')[0]

    if sgrp in ['triclinic', *[str(num) for num in range(1, 3)]]:
        args = [params[key] for key in ['a', 'b', 'c', 'alpha', 'beta', 'gamma']]

    elif sgrp in ['monoclinic', *[str(num) for num in range(3, 16)]]:
        args = [params[key] for key in ['a', 'b', 'c', 'beta']]

    elif sgrp in ['orthorhombic', *[str(num) for num in range(16, 75)]]:
        args = [params[key] for key in ['a', 'b', 'c']]

    elif sgrp in ['tetragonal', *[str(num) for num in range(75, 143)]]:
        args = [params[key] for key in ['a', 'c']]

    elif sgrp in ['trigonal', *[str(num) for num in range(143, 168)]]:
        args = [params[key] for key in ['a', 'c']]

    elif sgrp in ['hexagonal', *[str(num) for num in range(168, 195)]]:
        args = [params[key] for key in ['a', 'c']]

    elif sgrp in ['cubic', *[str(num) for num in range(195, 231)]]:
        args = [params[key] for key in ['a']]

    return args

In [92]:
get_sym_args(calcite.lattice.space_group, calcite.lattice._parameters)

[4.988, 17.061]

In [4]:
elements_folder = f'output_tiff_scan2D_{scanid}_xs_sum8ch\\'

element_maps = {}
for element in os.listdir(f'{filedir}{elements_folder}'):
    el_im = io.imread(f'{filedir}{elements_folder}{element}')
    element_maps[str(element[:-5])] = el_im

In [23]:
key = 'detsum_Ni_K'
#interactive_dynamic_2d_plot(proc_map, range(121), range(121), img_vmin=10, img_vmax=100, display_map=element_maps['detsum_Pt_L'], map_vmin=1e4, map_vmax=1.1e4)
interactive_dynamic_2d_plot(proc_map, range(proc_map.shape[0]), range(proc_map.shape[1]), display_map=element_maps[key], display_title=key)

In [14]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5), dpi=200)

im = ax.imshow(med_img)
fig.colorbar(im, ax=ax)

fig.show()

In [5]:
ai = pyFAI.load(f'{filedir}scan149256_dexela.poni')
ai.energy = 15
tth_resolution = 0.02 # Degrees
chi_resolution = 0.05 # Degrees

res = ai.integrate2d_ng(proc_map.reshape(proc_map.shape[0] * proc_map.shape[1], proc_map.shape[2], proc_map.shape[3])[0], 100, 100, unit='2th_deg')
tth, chi = res[1], res[2]
tth_num = int(np.abs(np.max(tth) - np.min(tth)) // tth_resolution)
chi_num = int(np.abs(np.max(chi) - np.min(chi)) // chi_resolution)
# The bounds of interpolation should be limited by the intrument resolution AND the original image size. Should not make up too much data

# Is there a better way I could do this?
res = ai.integrate2d_ng(proc_map.reshape(proc_map.shape[0] * proc_map.shape[1], proc_map.shape[2], proc_map.shape[3])[0], tth_num, chi_num, unit='2th_deg')
tth, chi = res[1], res[2]
extent = [tth[0], tth[-1], chi[0], chi[-1]]

# Calibrate the full map. Can I parallelize this??
calibrated_map = np.zeros((proc_map.shape[0] * proc_map.shape[1], chi_num, tth_num), dtype=(dtype))
for i, pixel in enumerate(proc_map.reshape(proc_map.shape[0] * proc_map.shape[1], proc_map.shape[2], proc_map.shape[3])):
    res, _, _ = ai.integrate2d_ng(pixel, tth_num, chi_num, unit='2th_deg')
    calibrated_map[i] = res
calibrated_map = calibrated_map.reshape(proc_map.shape[0], proc_map.shape[1], chi_num, tth_num)

In [11]:
key = 'detsum_Ni_K'
#interactive_dynamic_2d_plot(proc_map, range(121), range(121), img_vmin=10, img_vmax=100, display_map=element_maps['detsum_Pt_L'], map_vmin=1e4, map_vmax=1.1e4)
interactive_dynamic_2d_plot(calibrated_map, tth, chi, display_map=element_maps[key], display_title=key)

In [8]:
fig, ax = plt.subplots(2, 1, figsize=(10, 5), dpi=200, sharex=True)

max_img = np.max(proc_map, axis=(0, 1)).astype(dtype)
min_img = np.min(proc_map, axis=(0, 1)).astype(dtype)
ref_img = max_img - min_img

ref_img.astype(np.float16)[ref_img < 0] = 0

res = ai.integrate2d_ng(ref_img, 4096, 360, unit='2th_deg')
tth, chi = res[1], res[2]
xrd_int = res[0]

im = ax[0].imshow(res[0], extent=[tth[0], tth[-1], chi[0], chi[-1]], aspect='auto', norm=LogNorm(vmin=100))
fig.colorbar(im, ax=ax)
ax[0].set_ylabel('Azimuthal Angle, χ [°]')
ax[1].set_xlabel('Scattering Angle, 2θ [°]')
_, res = ai.integrate1d_ng(ref_img, 4096, unit='2th_deg') # I really do not like that the 1D and 2D outputs are flipped
xrd_int = res
ax[1].plot(tth, res)

fig.show()

In [9]:
cif_dir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Literature\\CIF\\'''
hematite = Phase.fromCIF(f'{cif_dir}AMCSD\\Hematite_0017806.cif')
hematite.name = 'Hematite'

iron_alpha = Phase.fromCIF(f'{cif_dir}AMCSD\\Iron-alpha_0011214.cif')
iron_alpha.name = 'Iron-ferrite'

iron_beta = Phase.fromCIF(f'{cif_dir}AMCSD\\Iron-beta_0011215.cif')
iron_beta.name = 'Iron-beta'

iron_delta = Phase.fromCIF(f'{cif_dir}AMCSD\\Iron-delta_0011216.cif')
iron_delta.name = 'Iron-delta'

iron = Phase.fromCIF(f'{cif_dir}AMCSD\\Iron_0011146.cif')
iron.name = 'Iron-austenite'

cementite = Phase.fromCIF(f'{cif_dir}AMCSD\\Cementite_0013523.cif')
cementite.name = 'Cementite'

zincite = Phase.fromCIF(f'{cif_dir}AMCSD\\Zincite_0011555.cif')
zincite.name = 'Zincite'

In [10]:
size=200
bkg_spline = SplineModel(xknots=np.linspace(tth[0], tth[-1], 20))
params = bkg_spline.guess(median_filter(xrd_int, size=(size)), tth)
bkg_fit = bkg_spline.fit(median_filter(xrd_int, size=(size)), params, x=tth)
bkg = bkg_fit.best_fit

In [24]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

#bkg = rolling_ball(xrd_int, radius=50)
#bkg_spline = UnivariateSpline(tth, xrd_int)
#bkg_spline.set_smoothing_factor(6e7)
#bkg = bkg_spline(tth)

ax.plot(tth, xrd_int, label='exp')
ax.plot(tth, median_filter(xrd_int, size=(size)), label='median')
ax.plot(tth, bkg, label='bkg')
ax.legend()

fig.show()

In [11]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5), dpi=200)
colors = matplotlib.color_sequences['tab10']

norm_xrd_int = rescale_array(xrd_int - bkg, upper=100, arr_min=0)

phases = [hematite, iron, iron_alpha, iron_beta, iron_delta, cementite, zincite]
[phase.get_hkl_reflections(tth_range=(tth[0], tth[-1]), en=ai.energy*1e3) for phase in phases if phase.reflections is None];

xrd_plot = ax.plot(tth, norm_xrd_int, label='Composite XRD', c='k', zorder=(len(phases) + 1))
fig.subplots_adjust(right=0.75)

def update_max(LineCollection, phase, slider):
    seg = np.asarray(LineCollection.get_segments())
    seg[:, 1, 1] = slider.val * phase.reflections['int'] / np.min(phase.reflections['int'])
    return seg

lines = [xrd_plot]
slider_lst = []
update_lst = []

def update_factory(index):
    def update(val):
        lines[index + 1].set_segments(update_max(lines[index + 1], phases[index], slider_lst[index]))
        fig.canvas.draw_idle()
    return update

slider_vpos = np.linspace(0.8, 0.1, len(phases))
for i, phase in enumerate(phases):
    phase_intensities = phase.reflections['int']
    phase_scale = np.max(norm_xrd_int) / np.max(phase_intensities)
    phase_plot = ax.vlines(phase.reflections['tth'], ymin=0, ymax=0, color=colors[i], lw=2)
    lines.append(phase_plot)

    axphase = fig.add_axes([0.8, slider_vpos[i], 0.1, 0.03])
    axphase.set_title(phase.name, fontsize=10)
    phase_amp = Slider(
            ax=axphase,
            label='',
            valmin=0,
            valmax=100,
            valinit=0,
            initcolor='none',
            color = colors[i],
            handle_style={'edgecolor' : colors[i], 'size' : 8}
            )
    
    slider_lst.append(phase_amp)
    update_lst.append(update_factory(i))
    slider_lst[i].on_changed(update_lst[i])

ax.set_ylim(0)
ax.set_xlabel('Scattering Angle, 2θ [°]')
ax.set_title('Phase Selector')
ax.legend(fontsize=10)
fig.show()

In [12]:
test_map = ImageMap.

ValueError: maximum supported dimension for an ndarray is 32, found 61